Model: xgboost \\
Features: kenpom \\
Trainings sets:

In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cross_validation import StratifiedKFold, PredefinedSplit, cross_val_score
from sklearn.grid_search import GridSearchCV

In [2]:
import pandas as pd

In [41]:
import numpy as np

In [3]:
data = pd.DataFrame.from_csv("kenpom_team_ratings.csv")

In [350]:
tourney_data = pd.DataFrame.from_csv("march_madness_2016_data/TourneyCompactResults.csv")

In [193]:
teams = pd.DataFrame.from_csv("march_madness_2016_data/teams.csv")

In [101]:
seeds = pd.DataFrame.from_csv("march_madness_2016_data/TourneySeeds.csv")

In [1019]:
X = pd.DataFrame()
Y = []

In [1020]:
features_to_include = ['OffenseAdjOE',
                       'TempoAdjusted',
                       'OffenseeFG%',
                       'OffenseTO%',
                       'OffenseOR%',
                       '3P%',
                       '2P%']
features_to_include_both_teams = []
for feature in features_to_include:
    features_to_include_both_teams.append(feature + '_x')
    features_to_include_both_teams.append(feature + '_y')

In [1021]:
# Hold out seasons as cross-validation, currently 2015
X_cv = pd.DataFrame()
Y_cv = []

In [1022]:
for i, year in enumerate(years):
    year_teams = pd.DataFrame(teams_from_kenpom[i])
    year_teams.columns = ['Team', 'Team_ID']
    year_data = data[data['Year'] == int(year)]
    year_data['Team'] = year_data['Team'].apply(remove_seed)
    year_data = pd.merge(year_data, year_teams, on=['Team'])
    year_results = tourney_data.loc[str(year)]
    
    year_data = year_data.rename(columns={'Team_ID':'Wteam'})
    year_results = pd.merge(year_results, year_data, on='Wteam')
    year_data = year_data.rename(columns={'Wteam':'Lteam'})
    year_results = pd.merge(year_results, year_data, on='Lteam', how='outer')
    year_results = year_results[year_results['Wteam'] == year_results['Wteam']]
    
    # Temporary, should consider ways to deal with missing data
    year_results = year_results.dropna()
    
    y = year_results['Wscore'] - year_results['Lscore']
    year_results = year_results[features_to_include_both_teams]
    
    if year != 2015:
        X = X.append(year_results, ignore_index=True)
        Y += list(y)
    else:
        X_cv = X_cv.append(year_results, ignore_index=True)
        Y_cv += list(y)


c:\toolkits\anaconda2-4.2.0\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1023]:
import random

In [1024]:
# Randomize the order of the teams
switch_array = np.array([0] * (X.shape[0]/2) + [1] * (X.shape[0] - X.shape[0]/2))
np.random.shuffle(switch_array)
X_random = pd.DataFrame()
Y_random = [-y if x == 0 else y for x,y in zip(switch_array, Y)]
for feature in features_to_include:
    X_random[feature + '_x'] = X[feature + '_x'] * switch_array + X[feature + '_y'] * (1 - switch_array)
    X_random[feature + '_y'] = X[feature + '_x'] * (1 - switch_array) + X[feature + '_y'] * switch_array

switch_array = np.array([0] * (X_cv.shape[0]/2) + [1] * (X_cv.shape[0] - X_cv.shape[0]/2))
np.random.shuffle(switch_array)
X_cv_random = pd.DataFrame()
Y_cv_random = [-y if x == 0 else y for x,y in zip(switch_array, Y_cv)]
for feature in features_to_include:
    X_cv_random[feature + '_x'] = X_cv[feature + '_x'] * switch_array + X_cv[feature + '_y'] * (1 - switch_array)
    X_cv_random[feature + '_y'] = X_cv[feature + '_x'] * (1 - switch_array) + X_cv[feature + '_y'] * switch_array

In [1025]:
Y = Y_random
X = X_random
X_cv = X_cv_random
Y_cv = Y_cv_random

In [1059]:
xgb_ = XGBClassifier()

xgb_params = {
    'learning_rate':[0.1, 0.15],
    'n_estimators':[60, 70, 80],
    'min_child_weight':[0.1],
    'subsample':[0.1],
    'colsample_bytree':[1.0],
    'colsample_bylevel':[1.0],
    'gamma':[0],
    'max_depth':[1, 2, 4]
}

grid = GridSearchCV(xgb_, xgb_params, scoring='mean_squared_error', cv=None, verbose=1, n_jobs=8)
grid.fit(X, Y)
print grid.best_params_
print grid.best_score_

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  54 out of  54 | elapsed:  2.0min finished


{'colsample_bytree': 1.0, 'colsample_bylevel': 1.0, 'learning_rate': 0.1, 'min_child_weight': 0.1, 'n_estimators': 80, 'subsample': 0.1, 'max_depth': 2, 'gamma': 0}
-191.316725979


In [1060]:
best = grid.best_params_
d_train = xgb.DMatrix(X, label=Y)
a = xgb.cv(best, d_train, metrics=['rmse'], nfold=3, num_boost_round=100)
print a['test-rmse-mean'].min()
print a['test-rmse-mean'].argmin()

12.25279
36


In [1061]:
best_n_rounds = a['test-rmse-mean'].argmin()
model = xgb.train(best, d_train, num_boost_round=best_n_rounds)

In [1062]:
def spread_to_percent(spread):
    return 1/(1 + 10 ** (-spread / 15))

In [1063]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

In [1064]:
train_pred = xgb.DMatrix(X)
preds = spread_to_percent(model.predict(train_pred))
Y_ = [0 if y < 0 else 1 for y in Y]

In [1065]:
logloss(Y_, preds)

0.55615622472694126

In [1066]:
cv_pred = xgb.DMatrix(X_cv)
cv_preds = spread_to_percent(model.predict(cv_pred))
Y_cv_ = [0 if y < 0 else 1 for y in Y_cv]

In [1067]:
logloss(Y_cv_, cv_preds)

0.57836816304210403

In [1078]:
#zip(cv_preds, Y_cv_)
sum([1 if round(x) == y else 0 for x,y in zip(cv_preds, Y_cv_)]) / float(len(cv_preds))

0.6417910447761194

In [709]:
spellings = pd.DataFrame.from_csv("march_madness_2016_data/TeamSpellings.csv")

# Add names from kenpoms website
names_to_add = {'mississippi valley st.':'1290',
                'arkansas pine bluff':'1115',
                'arkansas little rock':'1114',
                'louisiana lafayette':'1418',
                'cal st. bakersfield':'1167',
                'illinois chicago':'1227',
                'texas a&m corpus chris':'1394'}

for name_to_add in names_to_add:
    spellings.ix[name_to_add,:] = names_to_add[name_to_add]

In [625]:
def remove_seed(name):
    split_name = name.split(" ")
    if split_name[-1].isdigit():
        return " ".join(split_name[:-1])
    return name

In [710]:
a = data['Team'].copy()
a.sort_values(axis=0)
a.unique()
b = pd.Series()
for i, a_ in enumerate(a):
    new = a_.split(" ")
    if new[-1].isdigit():
        new = " ".join(new[:-1])
        b.set_value(i, new)
    else:
        b.set_value(i, a_)
b = sorted(b.unique())

In [711]:
years = range(2002,2016)
teams_from_kenpom = [[] for _ in years]
teams_from_kaggle = [[] for _ in years]

In [712]:
for i, year in enumerate(years):
    year_seeds = list(seeds.loc[str(year)]['Team'])
    seeds_used = []
    for b_ in b:
        try:
            team_id = spellings.loc[b_.lower()]
            if int(team_id) in year_seeds and int(team_id) not in seeds_used:
                teams_from_kenpom[i].append([b_, int(team_id)])
                seeds_used.append(int(team_id))
        except KeyError:
            pass

In [713]:
for i, year in enumerate(years):
    year_seeds = list(seeds.loc[str(year)]['Team'])
    for seed in year_seeds:
        index = seed - 1101
        name = teams.iloc[index]
        teams_from_kaggle[i].append([name[0], int(seed)])

In [714]:
for i in xrange(len(years)):
    teams_from_kenpom[i].sort(key=lambda x: x[1])
    teams_from_kaggle[i].sort(key=lambda x: x[1])

In [765]:
# Compare kaggle names to kenpom names to ensure that all teams are correctly saved
"""i = 13
print len(teams_from_kenpom[i])
print len(teams_from_kaggle[i])
zip(teams_from_kenpom[i], teams_from_kaggle[i])"""

'i = 13\nprint len(teams_from_kenpom[i])\nprint len(teams_from_kaggle[i])\nzip(teams_from_kenpom[i], teams_from_kaggle[i])'